# Debug notebook
This notebook is only used to troubleshoot the S1 ARD workflow processor.

## S1 ARD workflow

In [1]:
import json
from utils.prepS1 import prepareS1

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    logger.info(f"{x}: {loaded_json[x]}")
    prepareS1(**loaded_json)

## S1 ARD job queue processor

In [2]:
import os
import logging
import rediswq

level = os.getenv("LOGLEVEL", "INFO").upper()
logging.basicConfig(format="%(asctime)s %(levelname)-8s %(name)s %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=level)

host = os.getenv("REDIS_SERVICE_HOST", "redis-master")
q = rediswq.RedisWQ(name="jobS1", host=host)

logger = logging.getLogger("worker")
logger.info(f"Worker with sessionID: {q.sessionID()}")
logger.info(f"Initial queue state: empty={q.empty()}")

2020-10-21 14:00:03 INFO     worker Worker with sessionID: db7afbf4-2c05-4895-a10b-bde6fd5b999b
2020-10-21 14:00:03 INFO     worker Initial queue state: empty=False


In [3]:
import datetime

while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        logger.info(f"Working on {itemstr}")

        start = datetime.datetime.now().replace(microsecond=0)
        process_scene(itemstr)
        q.complete(item)
        end = datetime.datetime.now().replace(microsecond=0)
        logger.info(f"Total processing time {end - start}")
    else:
        logger.info("Waiting for work")

2020-10-21 14:00:39 INFO     worker Working on {"in_scene": "S1A_IW_GRDH_1SDV_20200504T064007_20200504T064042_032411_03C0C7_FE9D", "s3_bucket": "public-eo-data", "s3_dir": "luigi/fiji/sentinel_1/", "ext_dem": "common_sensing/ancillary_products/SRTM1Sec/SRTM30_Fiji_E.tif"}
2020-10-21 14:00:39 INFO     root S1A_IW_GRDH_1SDV_20200504T064007_20200504T064042_032411_03C0C7_FE9D.SAFE S1A_IW_GRDH_1SDV_20200504T064007 Starting
2020-10-21 14:00:39 INFO     root S1A_IW_GRDH_1SDV_20200504T064007_20200504T064042_032411_03C0C7_FE9D.SAFE S1A_IW_GRDH_1SDV_20200504T064007 DOWNLOADING via ASF
2020-10-21 14:00:39 INFO     root fetching: https://api.daac.asf.alaska.edu/services/search/param?granule_list=S1A_IW_GRDH_1SDV_20200504T064007_20200504T064042_032411_03C0C7_FE9D&output=csv
2020-10-21 14:00:40 INFO     root logging into asf
2020-10-21 14:00:44 INFO     root successfully logged into asf
2020-10-21 14:01:45 INFO     root Extracting ASF scene: /tmp/data/intermediate/S1A_IW_GRDH_1SDV_20200504T064007_tm

not boo


2020-10-21 14:34:59 INFO     worker Total processing time 0:34:20


In [4]:
logger.info("Queue empty!")

2020-10-21 14:34:59 INFO     worker Queue empty!


---